In [14]:
import numpy as np
import colorama as col
from random import randint, choice

cols = [
    col.Back.RESET,
    col.Back.WHITE,
    col.Back.BLUE,
    col.Back.MAGENTA,
    col.Back.YELLOW,
    col.Back.GREEN,
    col.Back.RED,
]

class MagicCube:
    def __init__(self, vis_bais=0):
        self.state = np.array([[i + 1] * 9 for i in range(6)], dtype=np.int32)
        self.vis_bais = vis_bais
        # fmt:off
        self.action = {
            "R" : (1, 0),
            "R'": (1, 1),
            "U" : (0, 0),
            "U'": (0, 1),
            "L" : (4, 0),
            "L'": (4, 1),
            "D" : (3, 0),
            "D'": (3, 1),
            "F":  (5, 0),
            "F'": (5, 1),
        }
        # fmt:on

    def rotate(self, i, dir=0):
        i_ = ((i - 2) % 6, (i - 1) % 6, (i + 1) % 6, (i + 2) % 6)
        if (i % 2 + dir) % 2:
            # fmt:off
            t                        = self.state[i_[3], 0:3  ].copy()
            self.state[i_[3], 0:3  ] = self.state[i_[2], 0:7:3]
            self.state[i_[2], 0:7:3] = self.state[i_[1], 6:   ]
            self.state[i_[1], 6:   ] = self.state[i_[0], 2::3 ]
            self.state[i_[0], 2::3 ] = t
            # fmt:on
            self.state[i] = np.rot90(self.state[i].reshape((3, 3)), 1).reshape(-1)
        else:
            # fmt:off
            t                        = self.state[i_[0], 2::3 ].copy()
            self.state[i_[0], 2::3 ] = self.state[i_[1], 6:   ]
            self.state[i_[1], 6:   ] = self.state[i_[2], 0:7:3]
            self.state[i_[2], 0:7:3] = self.state[i_[3], 0:3  ]
            self.state[i_[3], 0:3  ] = t
            # fmt:on
            self.state[i] = np.rot90(self.state[i].reshape((3, 3)), -1).reshape(-1)

    def face(self, i):
        i = (i + self.vis_bais) % 6
        if i % 2:
            return self.state[i].reshape((3, 3)).T
        else:
            return self.state[i].reshape((3, 3))

    def vis_face(self, i):
        t = self.face(i)
        for l in range(len(t)):
            print("".join([cols[i] + "  " for i in t[l]]))

    def expansion(self):
        z = np.zeros((3, 3), dtype=np.int32)
        f = [self.face(i) for i in range(6)]
        return np.hstack(
            (
                np.vstack((f[0], f[1], z, z)),
                np.vstack((z, f[2], f[3], z)),
                np.vstack((z, z, f[4], f[5])),
            )
        )

    def vis_expansion(self):
        t = self.expansion()
        for l in range(len(t)):
            print("".join([cols[i] + "  " for i in t[l]]))

    def execute(self, acts):
        if isinstance(acts, str):
            acts = acts.split(" ")
        for i in acts:
            self.rotate(*self.action[i])

    def shuffle(self):
        seq = []
        for i in range(randint(3, 10)):
            seq.append(choice(list(self.action.keys())))
        self.execute(seq)
        return seq


c = MagicCube(4)
c.execute("U R U' R'")
c.vis_expansion()
